In [ ]:
!pip3 install sqlalchemy

In [1]:
import sqlalchemy as db
import pandas as pd

In [3]:
sqlite_db = {'drivername': 'sqlite', 'database': 'data/weather_repo.sqlite'}

# if file in subfolder, don't start string for subfolder with "/"
db_conn = db.engine.url.URL(**sqlite_db)
engine = db.create_engine(db_conn) # for database connections
print("Connected to Database {}:".format(engine))

connection = engine.connect() # for queries
metadata = db.MetaData()


Connected to Database Engine(sqlite:///data/weather_repo.sqlite):


UnboundExecutionError: MetaData object is not bound to an Engine or Connection.  Execution can not proceed without a database to execute against.

## Create a Table as Data Structure with Data Type

In [16]:
# Create table stationen

station = db.Table('stationen', metadata,
db.Column('S_ID', db.Integer()),
db.Column('Standort', db.String(255), nullable=False),
db.Column('Geo_Breite', db.Float(), nullable=False),
db.Column('Geo_Laenge', db.Float(), nullable=False),
db.Column('Hoehe', db.Integer(), nullable=False),
db.Column('Betreiber', db.String(255), nullable=False),
db.Column('PLZ_matched', db.String(5), nullable=False),
db.Column('Ort_matched', db.String(255), nullable=False),
db.Column('Latitude_matched', db.Float(), nullable=False),
db.Column('Longitude_matched', db.Float(), nullable=False),
extend_existing=True
)

metadata.create_all(engine) #Creates the table

In [128]:
# Create table wettermessung or table temp

table_name=""

if not table_name:
    table_name = input("Name of new table for Wettermessungen? ")

messung = db.Table(table_name, metadata,
db.Column('Stations_ID', db.Integer(), nullable=False, primary_key=True),
db.Column('Datum', db.DateTime(), nullable=False, primary_key=True),
db.Column('Qualitaet', db.Integer(), nullable=True),
db.Column('Min_5cm', db.Float(), nullable=True),
db.Column('Min_2m', db.Float(), nullable=True),
db.Column('Mittel_2m', db.Float(), nullable=True),
db.Column('Max_2m', db.Float(), nullable=True),
db.Column('Relative_Feuchte', db.Float(), nullable=True),
db.Column('Mittel_Windstaerke', db.Float(), nullable=True),
db.Column('Max_Windgeschwindigkeit', db.Float(), nullable=True),
db.Column('Sonnenscheindauer', db.Float(), nullable=True),
db.Column('Mittel_Bedeckungsgrad', db.Float(), nullable=True),
db.Column('Niederschlagshoehe', db.Float(), nullable=True),
db.Column('Mittel_Luftdruck', db.Float(), nullable=True),
extend_existing=True
)

metadata.create_all(engine) #Creates the table

""" Example data
Stations_ID; Datum;      Qualitaet; Min_5cm; Min_2m; Mittel_2m; Max_2m; Relative_Feuchte; Mittel_Windstaerke; Max_Windgeschwindigkeit; Sonnenscheindauer; Mittel_Bedeckungsgrad; Niederschlagshoehe; Mittel_Luftdruck; 
102;         2017-12-19; 3;          ;       5,3;    6;         7,1;    96,29;            6,6;                11,6;                    0;                 ;                      ;                   1026,28;
"""

messung

Name of new table for Wettermessungen? temp


Table('temp', MetaData(bind=None), Column('Stations_ID', Integer(), table=<temp>, nullable=False), Column('Datum', DateTime(), table=<temp>, nullable=False), Column('Qualitaet', Integer(), table=<temp>), Column('Min_5cm', Float(), table=<temp>), Column('Min_2m', Float(), table=<temp>), Column('Mittel_2m', Float(), table=<temp>), Column('Max_2m', Float(), table=<temp>), Column('Relative_Feuchte', Float(), table=<temp>), Column('Mittel_Windstaerke', Float(), table=<temp>), Column('Max_Windgeschwindigkeit', Float(), table=<temp>), Column('Sonnenscheindauer', Float(), table=<temp>), Column('Mittel_Bedeckungsgrad', Float(), table=<temp>), Column('Niederschlagshoehe', Float(), table=<temp>), Column('Mittel_Luftdruck', Float(), table=<temp>), schema=None)

## Import Data into Tables

In [96]:
# into Table "stationen"

df = pd.read_pickle("data/stat.pkl")
#df.reset_index(inplace=True)
#df.rename(columns={'index':'Index'}, inplace =True)
df.head()

,S_ID,Standort,Geo_Breite,Geo_Laenge,Hoehe,Betreiber,Plz_matched,Ort_matched,Latitude_matched,Longitude_matched
0,102,Leuchtturm Alte Weser,53.863,8.127,32,DWD,26486,Nordseebad Wangerooge,7.91141,53.7896
1,164,Angermünde,53.032,13.991,54,DWD,16278,Angermünde,14.01040,53.0547
2,183,Arkona,54.679,13.434,42,DWD,18556,Dranske,13.31760,54.6303
3,232,Augsburg,48.425,10.942,461,DWD,86169,Augsburg,10.90450,48.4209
4,282,Bamberg,49.874,10.921,240,DWD,96050,Bamberg,10.93140,49.8809


In [97]:
print("Connected to Database {}:".format(engine))
df.to_sql('stationen', con=engine, if_exists='append', index=False)

Connected to Database Engine(sqlite:///data/weather_repo.sqlite):


In [129]:
# Into Table "wettermessung" or Table "temp"

df = pd.read_pickle("data/wett.pkl")
#df.reset_index(inplace=True)
#df.rename(columns={'index':'Index'}, inplace =True)
df.head()

,Stations_ID,Datum,Qualitaet,Min_5cm,Min_2m,Mittel_2m,Max_2m,Relative_Feuchte,Mittel_Windstaerke,Max_Windgeschwindigkeit,Sonnenscheindauer,Mittel_Bedeckungsgrad,Niederschlagshoehe,Mittel_Luftdruck
0,102,2017-12-27,3.0,NaN,4.0,5.0,5.7,87.63,9.7,16.3,0.000,NaN,NaN,981.17
1,102,2017-12-28,3.0,NaN,2.7,4.7,6.3,83.42,12.6,20.9,1.900,NaN,NaN,987.32
2,102,2017-12-29,3.0,NaN,1.8,3.0,4.5,86.83,13.2,20.5,4.533,NaN,NaN,994.75
3,102,2017-12-30,3.0,NaN,2.8,5.3,7.9,92.00,13.8,25.0,0.000,NaN,NaN,989.25
4,102,2017-12-31,3.0,NaN,6.1,8.3,11.0,93.38,11.4,20.1,0.000,NaN,NaN,991.57


In [130]:
df.dtypes


Stations_ID                         int64
Datum                      datetime64[ns]
Qualitaet                         float64
Min_5cm                           float64
Min_2m                            float64
Mittel_2m                         float64
Max_2m                            float64
Relative_Feuchte                  float64
Mittel_Windstaerke                float64
Max_Windgeschwindigkeit           float64
Sonnenscheindauer                 float64
Mittel_Bedeckungsgrad             float64
Niederschlagshoehe                float64
Mittel_Luftdruck                  float64
dtype: object

In [131]:
max(df["Datum"])

Timestamp('2019-06-29 00:00:00')

In [132]:
#df.drop( df.loc["Stations_ID"]==15000, inplace=True)

#df[df.loc["Stations_ID"]==15000,:]

df["Datum"] == "2019-06-29"




0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
41327    False
41328    False
41329    False
41330    False
41331    False
41332    False
41333    False
41334    False
41335    False
41336    False
41337    False
41338    False
41339    False
41340    False
41341    False
41342    False
41343    False
41344    False
41345    False
41346    False
41347    False
41348    False
41349    False
41350    False
41351    False
41352    False
41353    False
41354    False
41355    False
41356     True
Name: Datum, Length: 41357, dtype: bool

In [116]:
df = df[df["Datum"] != "2019-06-29"]

In [133]:
messung.fullname


'temp'

In [135]:
print("Connected to Database {}:".format(engine))

table_name = messung.fullname
#table_name = "temp"

df.to_sql(table_name, con=engine, if_exists='replace', index=False)

Connected to Database Engine(sqlite:///data/weather_repo.sqlite):


## Inspect Tables and Table structure

In [136]:
inspector = db.inspect(engine)

for table_name in inspector.get_table_names():
    print(table_name)
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'])

stationen
Column: S_ID
Column: Standort
Column: Geo_Breite
Column: Geo_Laenge
Column: Hoehe
Column: Betreiber
Column: Plz_matched
Column: Ort_matched
Column: Latitude_matched
Column: Longitude_matched
temp
Column: Stations_ID
Column: Datum
Column: Qualitaet
Column: Min_5cm
Column: Min_2m
Column: Mittel_2m
Column: Max_2m
Column: Relative_Feuchte
Column: Mittel_Windstaerke
Column: Max_Windgeschwindigkeit
Column: Sonnenscheindauer
Column: Mittel_Bedeckungsgrad
Column: Niederschlagshoehe
Column: Mittel_Luftdruck
wettermessung
Column: Stations_ID
Column: Datum
Column: Qualitaet
Column: Min_5cm
Column: Min_2m
Column: Mittel_2m
Column: Max_2m
Column: Relative_Feuchte
Column: Mittel_Windstaerke
Column: Max_Windgeschwindigkeit
Column: Sonnenscheindauer
Column: Mittel_Bedeckungsgrad
Column: Niederschlagshoehe
Column: Mittel_Luftdruck


In [120]:
m = db.MetaData()
m.reflect(engine)
for table in m.tables.values():
    print(table.name)
    for column in table.c:
        print("Column: %s" % column.name)

stationen
Column: S_ID
Column: Standort
Column: Geo_Breite
Column: Geo_Laenge
Column: Hoehe
Column: Betreiber
Column: Plz_matched
Column: Ort_matched
Column: Latitude_matched
Column: Longitude_matched
temp
Column: Stations_ID
Column: Datum
Column: Qualitaet
Column: Min_5cm
Column: Min_2m
Column: Mittel_2m
Column: Max_2m
Column: Relative_Feuchte
Column: Mittel_Windstaerke
Column: Max_Windgeschwindigkeit
Column: Sonnenscheindauer
Column: Mittel_Bedeckungsgrad
Column: Niederschlagshoehe
Column: Mittel_Luftdruck
wettermessung
Column: Stations_ID
Column: Datum
Column: Qualitaet
Column: Min_5cm
Column: Min_2m
Column: Mittel_2m
Column: Max_2m
Column: Relative_Feuchte
Column: Mittel_Windstaerke
Column: Max_Windgeschwindigkeit
Column: Sonnenscheindauer
Column: Mittel_Bedeckungsgrad
Column: Niederschlagshoehe
Column: Mittel_Luftdruck


## Delete single table

In [79]:
station = db.Table('station', 
                   metadata,
                   autoload=True,
                   autoload_with=engine)
station.drop(engine, checkfirst = True)

OperationalError: (sqlite3.OperationalError) no such table: station
[SQL: 
DROP TABLE station]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [121]:
metadata.tables.keys()

dict_keys(['stationen', 'wettermessung'])

In [88]:
sql = 'DROP TABLE IF EXISTS station;'
result = engine.execute(sql)

## Query Tables of Database

**Table "stationen"**

In [122]:
print("Content of '{}'': ".format(sqlite_db["database"]))

station = db.Table('stationen', 
                   metadata,
                   autoload=True,
                   autoload_with=engine)

results = connection.execute(db.select([station])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

Content of 'data/weather_repo.sqlite'': 


,S_ID,Standort,Geo_Breite,Geo_Laenge,Hoehe,Betreiber,Plz_matched,Ort_matched,Latitude_matched,Longitude_matched
0,102,Leuchtturm Alte Weser,53.863,8.127,32,DWD,26486,Nordseebad Wangerooge,7.91141,53.7896
1,164,Angermünde,53.032,13.991,54,DWD,16278,Angermünde,14.01040,53.0547
2,183,Arkona,54.679,13.434,42,DWD,18556,Dranske,13.31760,54.6303
3,232,Augsburg,48.425,10.942,461,DWD,86169,Augsburg,10.90450,48.4209


**Table "wettermessung"**

In [125]:
print("Content of '{}'': ".format(sqlite_db["database"]))

station = db.Table('wettermessung', 
                   metadata,
                   autoload=True,
                   autoload_with=engine)

results = connection.execute(db.select([station])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

Content of 'data/weather_repo.sqlite'': 


,Stations_ID,Datum,Qualitaet,Min_5cm,Min_2m,Mittel_2m,Max_2m,Relative_Feuchte,Mittel_Windstaerke,Max_Windgeschwindigkeit,Sonnenscheindauer,Mittel_Bedeckungsgrad,Niederschlagshoehe,Mittel_Luftdruck
0,102,2017-12-27,3.0,NaN,4.0,5.0,5.7,87.63,9.7,16.3,0.000,NaN,NaN,981.17
1,102,2017-12-28,3.0,NaN,2.7,4.7,6.3,83.42,12.6,20.9,1.900,NaN,NaN,987.32
2,102,2017-12-29,3.0,NaN,1.8,3.0,4.5,86.83,13.2,20.5,4.533,NaN,NaN,994.75
3,102,2017-12-30,3.0,NaN,2.8,5.3,7.9,92.00,13.8,25.0,0.000,NaN,NaN,989.25


In [124]:
df.dtypes

Stations_ID                         int64
Datum                      datetime64[ns]
Qualitaet                         float64
Min_5cm                           float64
Min_2m                            float64
Mittel_2m                         float64
Max_2m                            float64
Relative_Feuchte                  float64
Mittel_Windstaerke                float64
Max_Windgeschwindigkeit           float64
Sonnenscheindauer                 float64
Mittel_Bedeckungsgrad             float64
Niederschlagshoehe                float64
Mittel_Luftdruck                  float64
dtype: object

In [127]:
df["Datum"].max()

Timestamp('2019-06-28 00:00:00')

**Table "temp"**

In [138]:
print("Content of '{}'': ".format(sqlite_db["database"]))

station = db.Table('temp', 
                   metadata,
                   autoload=True,
                   autoload_with=engine)

results = connection.execute(db.select([station])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

Content of 'data/weather_repo.sqlite'': 


,Stations_ID,Datum,Qualitaet,Min_5cm,Min_2m,Mittel_2m,Max_2m,Relative_Feuchte,Mittel_Windstaerke,Max_Windgeschwindigkeit,Sonnenscheindauer,Mittel_Bedeckungsgrad,Niederschlagshoehe,Mittel_Luftdruck
0,102,2017-12-27,3.0,NaN,4.0,5.0,5.7,87.63,9.7,16.3,0.000,NaN,NaN,981.17
1,102,2017-12-28,3.0,NaN,2.7,4.7,6.3,83.42,12.6,20.9,1.900,NaN,NaN,987.32
2,102,2017-12-29,3.0,NaN,1.8,3.0,4.5,86.83,13.2,20.5,4.533,NaN,NaN,994.75
3,102,2017-12-30,3.0,NaN,2.8,5.3,7.9,92.00,13.8,25.0,0.000,NaN,NaN,989.25


In [137]:
df.dtypes

Stations_ID                         int64
Datum                      datetime64[ns]
Qualitaet                         float64
Min_5cm                           float64
Min_2m                            float64
Mittel_2m                         float64
Max_2m                            float64
Relative_Feuchte                  float64
Mittel_Windstaerke                float64
Max_Windgeschwindigkeit           float64
Sonnenscheindauer                 float64
Mittel_Bedeckungsgrad             float64
Niederschlagshoehe                float64
Mittel_Luftdruck                  float64
dtype: object

In [139]:
df["Datum"].max()

Timestamp('2019-06-29 00:00:00')

## Pre-Implementation

In [5]:
sqlite_db = {'drivername': 'sqlite', 'database': 'data/weather_data.sqlite'}

# if file in subfolder, don't start string for ubfolder with "/"
db_conn = db.engine.url.URL(**sqlite_db)
engine = db.create_engine(db_conn)
engine

Engine(sqlite:///data/weather_data.sqlite)

In [6]:
connection = engine.connect()
metadata = db.MetaData()
emp = db.Table('emp', metadata,
db.Column('Id', db.Integer()),
db.Column('name', db.String(255), nullable=False),
db.Column('salary', db.Float(), default=100.0),
db.Column('active', db.Boolean(), default=True)
)
metadata.create_all(engine) #Creates the table

In [9]:
query = db.insert(emp).values(Id=1, name='naveen', salary=60000.00,
active=True) 
ResultProxy = connection.execute(query)

In [11]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,Id,name,salary,active
0,1,naveen,60000.0,True
1,1,naveen,60000.0,True


## Insert data

**fast bulk insert from dataframe into sqlite**  

https://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy  
    
https://www.codementor.io/bruce3557/graceful-data-ingestion-with-sqlalchemy-and-pandas-pft7ddcy6  

In [19]:
#Inserting many records at ones
query = db.insert(station)
values_list = [
    {'S_ID': 2,
     'Standort': "",
     'Geo_Breite': 0.0,
     'Geo_Laenge': 0.0,
     'Hoehe': 0,
     'Betreiber': "",
     'PLZ_matched': "",
     'Ort_matched': "",
     'Latitude_matched': 0.0,
     'Longitude_matched': 0.0} 
]

ResultProxy = connection.execute(query,values_list)